# Foundry IQ: Grounding with Azure AI Search

> **Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub
> **Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)
> **© 2025 Ozgur Guler. All rights reserved.**

---

This notebook demonstrates how to ground an Azure AI Foundry agent with an existing Azure AI Search index.

## What is Grounding?

Grounding connects your agent to proprietary data sources, enabling:
- **Retrieval-Augmented Generation (RAG)** - Agent retrieves relevant documents before responding
- **Accurate answers** based on your indexed content
- **Reduced hallucination** by anchoring responses to real data

## Configuration

We'll connect to the following AI Search index:
- **AI Search Service**: `chatops`
- **Index Name**: `imf_baseline`

## Prerequisites

1. **Azure CLI authenticated**: Run `az login`
2. **Azure AI Foundry project**: From previous sections
3. **Azure AI Search service**: With an existing index
4. **Connection configured**: Between Foundry project and AI Search service

---

## Section 1: Setup and Configuration

In [ ]:
# Install required packages
!pip install azure-ai-projects --pre --quiet
!pip install azure-ai-agents --pre --quiet
!pip install azure-identity python-dotenv --quiet

In [ ]:
import os
from dotenv import load_dotenv

# Load environment from parent directory
load_dotenv("../.env")

# Foundry Configuration
FOUNDRY_ACCOUNT = os.getenv("FOUNDRY_ACCOUNT_NAME", "ozgurguler-7212-resource")
PROJECT_NAME = os.getenv("FOUNDRY_PROJECT_NAME", "ozgurguler-7212")
PROJECT_ENDPOINT = f"https://{FOUNDRY_ACCOUNT}.services.ai.azure.com/api/projects/{PROJECT_NAME}"

# AI Search Configuration
AI_SEARCH_SERVICE = os.getenv("AI_SEARCH_SERVICE", "chatops")
AI_SEARCH_ENDPOINT = f"https://{AI_SEARCH_SERVICE}.search.windows.net"
AI_SEARCH_INDEX = os.getenv("AI_SEARCH_INDEX", "imf_baseline")

# Model Configuration
CHAT_MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-5-nano")

# Agent Configuration
AGENT_NAME = "imf-grounded-agent"

print(f"Project Endpoint: {PROJECT_ENDPOINT}")
print(f"AI Search Endpoint: {AI_SEARCH_ENDPOINT}")
print(f"AI Search Index: {AI_SEARCH_INDEX}")
print(f"Model: {CHAT_MODEL}")

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# Initialize the client
credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

print("AIProjectClient initialized successfully")

---

## Section 2: Verify AI Search Connection

First, let's check if a connection to the AI Search service exists in the project.

In [ ]:
from azure.ai.projects.models import ConnectionType

# List all connections
print("Checking project connections...\n")

try:
    connections = client.connections.list()
    
    search_connections = []
    for conn in connections:
        print(f"Connection: {conn.name}")
        print(f"  Type: {conn.connection_type}")
        print(f"  Target: {getattr(conn, 'target', 'N/A')}")
        print()
        
        if 'search' in str(conn.connection_type).lower() or 'search' in conn.name.lower():
            search_connections.append(conn)
    
    if search_connections:
        print(f"✅ Found {len(search_connections)} AI Search connection(s)")
    else:
        print("⚠️  No AI Search connections found - you may need to create one")
        
except Exception as e:
    print(f"Error listing connections: {e}")

In [ ]:
# Try to get the default AI Search connection
print("Getting default AI Search connection...")

try:
    ai_search_connection = client.connections.get_default(ConnectionType.AZURE_AI_SEARCH)
    AI_SEARCH_CONNECTION_ID = ai_search_connection.id
    
    print(f"\n✅ Default AI Search connection found!")
    print(f"  Name: {ai_search_connection.name}")
    print(f"  ID: {ai_search_connection.id}")
    print(f"  Target: {getattr(ai_search_connection, 'target', 'N/A')}")
    
except Exception as e:
    print(f"\n⚠️  No default AI Search connection: {e}")
    print("\nYou need to create a connection. See Section 2b below.")
    AI_SEARCH_CONNECTION_ID = None

### Section 2b: Create AI Search Connection (if needed)

If no connection exists, create one using the Azure ML SDK.

In [ ]:
# Create AI Search connection if not found
CREATE_CONNECTION = False  # Set to True to create

if CREATE_CONNECTION and AI_SEARCH_CONNECTION_ID is None:
    print("Creating AI Search connection...")
    
    try:
        from azure.ai.ml import MLClient
        from azure.ai.ml.entities import AzureAISearchConnection
        
        # Get ML client
        ml_client = MLClient(
            credential=credential,
            subscription_id=os.getenv("AZURE_SUBSCRIPTION_ID"),
            resource_group_name=os.getenv("AZURE_RESOURCE_GROUP", "rg-ozgurguler-7212"),
            workspace_name=PROJECT_NAME
        )
        
        # Create connection (using AAD auth - no API key)
        connection = AzureAISearchConnection(
            name=f"{AI_SEARCH_SERVICE}-connection",
            endpoint=AI_SEARCH_ENDPOINT,
            api_key=None  # Use AAD authentication
        )
        
        ml_client.connections.create_or_update(connection)
        print(f"✅ Created connection: {connection.name}")
        
        # Refresh connection ID
        ai_search_connection = client.connections.get_default(ConnectionType.AZURE_AI_SEARCH)
        AI_SEARCH_CONNECTION_ID = ai_search_connection.id
        
    except Exception as e:
        print(f"❌ Error creating connection: {e}")
        print("\nAlternatively, create the connection via Azure Portal:")
        print("1. Go to Azure AI Foundry portal")
        print("2. Select your project")
        print("3. Go to Settings > Connections")
        print("4. Add a new Azure AI Search connection")
else:
    if AI_SEARCH_CONNECTION_ID:
        print("AI Search connection already exists")
    else:
        print("Connection creation skipped (CREATE_CONNECTION = False)")

---

## Section 3: Verify the Search Index

Let's verify the `imf_baseline` index exists and check its schema.

In [ ]:
import subprocess
import json

# Get AI Search service info
print(f"Checking index '{AI_SEARCH_INDEX}' on '{AI_SEARCH_SERVICE}'...\n")

# Use REST API to check index (requires proper auth)
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import requests

try:
    # Get token for AI Search
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(),
        "https://search.azure.com/.default"
    )
    token = token_provider()
    
    # Check index exists
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(
        f"{AI_SEARCH_ENDPOINT}/indexes/{AI_SEARCH_INDEX}?api-version=2024-07-01",
        headers=headers
    )
    
    if response.status_code == 200:
        index_info = response.json()
        print(f"✅ Index '{AI_SEARCH_INDEX}' found!\n")
        print(f"Fields:")
        for field in index_info.get('fields', [])[:10]:  # Show first 10 fields
            print(f"  - {field['name']} ({field['type']})")
        if len(index_info.get('fields', [])) > 10:
            print(f"  ... and {len(index_info['fields']) - 10} more fields")
    else:
        print(f"⚠️  Index not found or access denied: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Error checking index: {e}")
    print("\nNote: You may need proper RBAC permissions on the AI Search service.")

---

## Section 4: Configure the Azure AI Search Tool

Now let's create the AI Search tool that will enable our agent to query the index.

In [ ]:
from azure.ai.agents.models import AzureAISearchTool, AzureAISearchQueryType

# Configure the Azure AI Search tool
print("Configuring Azure AI Search tool...\n")

if AI_SEARCH_CONNECTION_ID:
    ai_search_tool = AzureAISearchTool(
        index_connection_id=AI_SEARCH_CONNECTION_ID,
        index_name=AI_SEARCH_INDEX,
        query_type=AzureAISearchQueryType.SIMPLE,  # or SEMANTIC for better results
        top_k=5,  # Number of results to retrieve
        filter="",  # Optional OData filter
    )
    
    print(f"✅ AI Search tool configured:")
    print(f"  Connection ID: {AI_SEARCH_CONNECTION_ID}")
    print(f"  Index: {AI_SEARCH_INDEX}")
    print(f"  Query Type: SIMPLE")
    print(f"  Top K: 5")
else:
    print("❌ Cannot configure tool - no AI Search connection ID")
    print("Please create an AI Search connection first (Section 2b)")
    ai_search_tool = None

---

## Section 5: Create Grounded Agent

Create an agent that uses the AI Search tool for grounding.

In [ ]:
# Agent instructions optimized for grounding
GROUNDED_AGENT_INSTRUCTIONS = f"""
You are a helpful assistant that answers questions using the IMF (International Monetary Fund) knowledge base.

IMPORTANT RULES:
1. You MUST use the Azure AI Search tool to find relevant information before answering.
2. You MUST ground your answers in the retrieved documents.
3. If the search returns no relevant results, say "I couldn't find information about that in the knowledge base."
4. NEVER make up information - only use what you find in the search results.
5. Always cite your sources by mentioning which document the information came from.

The knowledge base contains: {AI_SEARCH_INDEX}

When responding:
- Be concise and accurate
- Quote relevant passages when helpful
- If asked about topics outside the knowledge base, politely redirect to what you can help with
"""

print("Agent Instructions:")
print(GROUNDED_AGENT_INSTRUCTIONS)

In [ ]:
# Create the grounded agent
print(f"Creating grounded agent: {AGENT_NAME}...\n")

if ai_search_tool:
    try:
        # Check if agent already exists
        try:
            existing_agent = client.agents.retrieve(agent_name=AGENT_NAME)
            print(f"Agent already exists: {existing_agent.name} (version: {existing_agent.version})")
            agent = existing_agent
        except:
            # Create new agent
            agent = client.agents.create_agent(
                model=CHAT_MODEL,
                name=AGENT_NAME,
                instructions=GROUNDED_AGENT_INSTRUCTIONS,
                tools=ai_search_tool.definitions,
                tool_resources=ai_search_tool.resources,
                temperature=0.1,  # Low temperature for factual responses
            )
            print(f"✅ Created grounded agent: {agent.name}")
            print(f"   ID: {agent.id}")
            print(f"   Model: {CHAT_MODEL}")
        
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        agent = None
else:
    print("❌ Cannot create agent - AI Search tool not configured")
    agent = None

---

## Section 6: Test the Grounded Agent

Let's test the agent with questions that should be answered from the IMF knowledge base.

In [ ]:
from azure.ai.agents.models import MessageRole, ListSortOrder

def ask_grounded_agent(question: str, agent, client) -> str:
    """Send a question to the grounded agent and get a response."""
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print("="*60)
    
    # Create a thread
    thread = client.agents.threads.create()
    
    # Send the message
    message = client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=question,
    )
    
    # Run the agent
    run = client.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent.id,
    )
    
    print(f"\nRun status: {run.status}")
    
    if run.status == "failed":
        print(f"Error: {run.last_error}")
        return None
    
    # Get the response
    messages = client.agents.messages.list(
        thread_id=thread.id,
        order=ListSortOrder.DESCENDING
    )
    
    for msg in messages.data:
        if msg.role == MessageRole.ASSISTANT:
            response_text = msg.content[0].text.value if msg.content else "No response"
            print(f"\nAgent Response:\n{response_text}")
            return response_text
    
    return None

In [ ]:
# Test questions for the IMF knowledge base
if agent:
    test_questions = [
        "What is the IMF's role in global economic stability?",
        "Summarize the key points from the latest available IMF report.",
        "What recommendations does the IMF provide for developing economies?",
    ]
    
    for question in test_questions:
        response = ask_grounded_agent(question, agent, client)
        print("\n")
else:
    print("Agent not available - please create it first")

---

## Section 7: Alternative - Using Prompt Agent with AI Search

You can also use the newer `PromptAgentDefinition` with the AI Search tool.

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

# Create agent using PromptAgentDefinition (newer API)
PROMPT_AGENT_NAME = "imf-grounded-prompt-agent"

if ai_search_tool and AI_SEARCH_CONNECTION_ID:
    try:
        prompt_agent = client.agents.create_version(
            agent_name=PROMPT_AGENT_NAME,
            definition=PromptAgentDefinition(
                model=CHAT_MODEL,
                instructions=GROUNDED_AGENT_INSTRUCTIONS,
                tools=ai_search_tool.definitions,
                tool_resources=ai_search_tool.resources,
            )
        )
        print(f"✅ Created prompt agent: {prompt_agent.name} (v{prompt_agent.version})")
        
    except Exception as e:
        print(f"Error: {e}")
        prompt_agent = None
else:
    print("AI Search tool not configured")
    prompt_agent = None

In [ ]:
# Test the prompt agent using conversations API
if prompt_agent:
    openai_client = client.get_openai_client()
    
    # Create conversation
    conversation = openai_client.conversations.create()
    print(f"Created conversation: {conversation.id}\n")
    
    # Test query
    test_query = "What are the main economic indicators discussed in the IMF baseline?"
    print(f"Query: {test_query}")
    
    response = openai_client.responses.create(
        input=test_query,
        conversation=conversation.id,
        extra_body={"agent": {"name": prompt_agent.name, "type": "agent_reference"}},
    )
    
    print(f"\nStatus: {response.status}")
    print(f"\nAgent Response:\n{response.output_text}")
else:
    print("Prompt agent not available")

---

## Section 8: Cleanup (Optional)

In [ ]:
# Delete agents
DELETE_AGENTS = False  # Set to True to delete

if DELETE_AGENTS:
    try:
        if agent:
            client.agents.delete_agent(agent.id)
            print(f"Deleted agent: {agent.id}")
    except Exception as e:
        print(f"Error deleting agent: {e}")
    
    try:
        if prompt_agent:
            client.agents.delete(agent_name=PROMPT_AGENT_NAME)
            print(f"Deleted prompt agent: {PROMPT_AGENT_NAME}")
    except Exception as e:
        print(f"Error deleting prompt agent: {e}")
else:
    print("Agent deletion skipped (DELETE_AGENTS = False)")

---

## Summary

### What We Built

1. **Connected to AI Search** - Verified/created connection to `chatops` service
2. **Verified the Index** - Confirmed `imf_baseline` index exists
3. **Configured AI Search Tool** - Created `AzureAISearchTool` with index
4. **Created Grounded Agent** - Agent that retrieves from the index
5. **Tested RAG Queries** - Verified grounding with real questions

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Grounding** | Anchoring agent responses in indexed content |
| **RAG** | Retrieval-Augmented Generation pattern |
| **AzureAISearchTool** | Tool that queries an AI Search index |
| **Connection** | Link between Foundry project and AI Search service |
| **Query Type** | SIMPLE (keyword) or SEMANTIC (AI-powered) |

### Best Practices

- **Use SEMANTIC query type** for better relevance
- **Set low temperature** (0-0.2) for factual responses
- **Instruct agent to cite sources** in the prompt
- **Use `tool_choice="required"`** to force search before answering
- **One index per agent** - use multi-agent for multiple indexes

### Troubleshooting

| Issue | Solution |
|-------|----------|
| No connection found | Create via Portal or SDK |
| Index not accessible | Check RBAC permissions |
| Agent hallucinating | Add stricter instructions, lower temperature |
| Empty responses | Verify index has data, check field mappings |

---

## Next Steps

Continue to `../07-logic-apps-as-mcp-server` for integration with Logic Apps.

---

<div align="center">

## License & Attribution

This notebook is part of the **Azure AI Foundry Demo Repository**

[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](../LICENSE)

**Original Author:** Ozgur Guler | AI Solution Leader, AI Innovation Hub

**Contact:** [ozgur.guler1@gmail.com](mailto:ozgur.guler1@gmail.com)

---

*If you use, modify, or distribute this work, you must provide appropriate credit to the original author as required by the [Apache License 2.0](../LICENSE).*

**Copyright © 2025 Ozgur Guler. All rights reserved.**

</div>